In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

LANGCHAIN_TRACING_V2 = os.environ.get("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = 'https://api.smith.langchain.com'
LANGCHAIN_API_KEY = os.environ.get("LANGCHAIN_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

llm = ChatOpenAI(model="qwen-max", temperature=0)

model_name = "C:\\Home\\Documents\\Projects\\models\\BAAI\\bge-large-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

c:\Apps\miniconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Vector Store

In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

vectorstore = QdrantVectorStore(
    client=client,
    collection_name="rag_from_scratch",
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# Generate Sub-questions

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

# Chain
generate_queries = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

In [5]:
question = "What is task decomposition for LLM agents?"

generate_queries.invoke(question)

['1. What are the key steps involved in task decomposition for LLM agents?',
 '2. How does task decomposition improve the performance of LLM agents?',
 '3. What are some common techniques or methods used for task decomposition in LLMs?']

# Final RAG Chain


### Chain 1

In [6]:
from operator import itemgetter

template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

prompt = ChatPromptTemplate.from_template(template)

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

questions = generate_queries.invoke({"question": question})
q_a_pairs = ""
for q in questions:
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

print(answer)

Task decomposition in Large Language Models (LLMs) is a crucial process for breaking down complex tasks into simpler, more manageable subtasks. Here are some common techniques or methods used for task decomposition in LLMs:

1. **Chain of Thought (CoT):**
   - **Description:** CoT involves instructing the LLM to "think step by step." This technique encourages the model to break down a complex task into a sequence of smaller, sequential steps.
   - **Benefits:** It helps in making the task more tractable and provides a clear, structured approach. CoT can also shed light on the model's reasoning process, making it easier to understand and evaluate.

2. **Tree of Thoughts (ToT):**
   - **Description:** ToT extends the Chain of Thought by allowing the LLM to explore multiple reasoning paths at each step. This creates a tree structure where different branches represent different lines of reasoning.
   - **Search Strategies:** The search through this tree can be conducted using strategies li

### Chain 2

In [7]:
# Answer each sub-question individually 

from langchain import hub

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag, sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.invoke(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries)

In [8]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

result = final_rag_chain.invoke({"context": context, "question": question})

print(result)

Task decomposition for LLM (Large Language Model) agents is a process that involves breaking down complex tasks into simpler, smaller, and more manageable steps. This approach enhances the model's ability to handle and solve intricate problems by making each component of the task more accessible and easier to process. 

Key techniques and methods used in task decomposition include:

1. **Chain of Thought (CoT)**: This method guides the LLM to break down a task into a sequence of simpler, sequential steps. By following a logical and step-by-step approach, the model can more effectively reason through the problem.

2. **Tree of Thoughts**: An extension of CoT, this technique explores multiple reasoning paths at each step, creating a tree-like structure. This allows the LLM to consider various possible solutions and navigate through them using search strategies like Breadth-First Search (BFS) or Depth-First Search (DFS).

3. **Simple Prompting and Task-Specific Instructions**: These invol

In [10]:
for question, answer in zip(questions, answers):
    print(f"Q: {question}\nA: {answer}")

Q: 1. What are the key steps involved in task decomposition for LLM agents?
A: The key steps in task decomposition for LLM agents include using techniques like Chain of Thought (CoT) to break down complex tasks into simpler, smaller steps. Another method is the Tree of Thoughts, which further expands on CoT by exploring multiple reasoning paths at each step, creating a tree structure that can be navigated through BFS or DFS. Additionally, task decomposition can be guided by simple prompts, task-specific instructions, or human inputs.
Q: 2. How does task decomposition improve the performance of LLM agents?
A: Task decomposition improves the performance of LLM agents by breaking down complex tasks into smaller, more manageable steps, which allows the model to think and act more effectively. This process, often guided by techniques like Chain of Thought or Tree of Thoughts, enables the LLM to explore multiple reasoning paths, enhancing its problem-solving capabilities. By simplifying task